In [13]:
import sys
sys.path.append('../../code/')
from utils.preprocess import read_json
from Ques_pair import Ques_pair, Type

In [14]:
ref_file = read_json("../../datasets/qald9plus/qald_9_plus_test_wikidata.json")
pred_file = read_json("/Users/mengshima/Downloads/mt5_11.json")

In [15]:
ref_ques_list = [ref for ref in ref_file["questions"]]
pred_ques_list = [pred for pred in pred_file["questions"]]

In [16]:
n = 5
for ref, pred in zip(ref_ques_list[:n], pred_ques_list[:n]):
    ques_pair = Ques_pair(ref, pred)
    ques_pair.print(print_answer=True)
    print()

id: 99
question: What is the time zone of Salt Lake City?
ref sparql: SELECT DISTINCT ?o1 WHERE { wd:Q23337 wdt:P421 ?o1 . }
pred sparql: SELECT DISTINCT ?uri WHERE { wd:Q220 wdt:P421 ?uri } 
ref_answer:
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3134980'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2212'}}]
pred_answer:
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6655'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6723'}}]


id: 98
question: Who killed Caesar?
ref sparql: SELECT DISTINCT ?o1 WHERE { wd:Q1048 wdt:P157 ?o1 . }
pred sparql: SELECT DISTINCT ?uri WHERE { wd:Q596 wdt:P61 ?uri . } 
ref_answer:
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q172248'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q207370'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q294846'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/

In [17]:
entity_f1_sum = 0
relation_f1_sum = 0
num_of_ques = len(ref_ques_list)

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    entity_f1_sum += ques_pair.calculate_entity_f1()
    relation_f1_sum += ques_pair.calculate_relation_f1()

macro_entity_f1 = entity_f1_sum / num_of_ques
macro_relation_f1 = relation_f1_sum / num_of_ques

In [18]:
macro_entity_f1

0.17107843137254902

In [19]:
macro_relation_f1

0.25232434640522877

In [20]:
# check reason of no answer
counter = 0

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.pred_answer==[]:
        ques_pair.print()
        counter += 1

print(str(counter) + " SPARQL queries don't give an answer")

# counter = sum([ques_pair.print() or 1 for ref, pred in zip(ref_ques_list, pred_ques_list) if (ques_pair := Ques_pair(ref, pred)).pred_answer == []])
# print(counter)


id: 49
question: Which frequent flyer program has the most airlines?
ref sparql: SELECT ?frequentflyer WHERE { ?airlines wdt:P31 wd:Q46970 . ?airlines p:P4446/ps:P4446 ?frequentflyer . } GROUP BY ?frequentflyer ORDER BY DESC(COUNT(?airlines)) LIMIT 1
pred sparql: SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q46970 } GROUP BY ?uri ORDER BY DESC(COUNT(DISTINCT ?uri) LIMIT 1

id: 42
question: Which countries have places with more than two caves?
ref sparql: SELECT DISTINCT ?uri WHERE { ?cave wdt:P31 wd:Q35509 ; wdt:P17 ?uri . } GROUP BY ?uri HAVING (COUNT(?cave) > 2)
pred sparql: SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q4022 . ?uri p_P2043 ?stmnode . ?stmnode psv_P2043 ?valuenode . ?valuenode wikibase_quantityAmount ?length . ?valuenode wikibase_quantityUnit ?unit . ?valuenode wikibase_quantityLowerBound ?lowerbound . ?valuenode wikibase_quantityUpperBound ?upperbound .

id: 68
question: Which country was Bill Gates born in?
ref sparql: SELECT DISTINCT ?uri WHERE { wd:Q5284 wdt:P19/w

In [21]:
# check correct answers
counter = 0

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.pred_answer==ques_pair.ref_answer:
        ques_pair.print()
        counter += 1

print(str(counter) + " questions are 100% correct")

id: 149
question: Which U.S. state has the highest population density?
ref sparql: SELECT ?uri { ?uri wdt:P31 wd:Q35657 . ?uri wdt:P1082 ?population . ?uri wdt:P2046 ?area . BIND((?population/?area) AS ?density ) } ORDER BY DESC(?density) LIMIT 1
pred sparql: SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q35657 ; wdt:P2046 ?area ; wdt:P1082 ?population . BIND(( ?population/ ?area) AS ?density) . } ORDER BY DESC( ?density) LIMIT 1

id: 117
question: Is Pamela Anderson a vegan?
ref sparql: ASK where { wd:Q83325 wdt:P1576 wd:Q83364}
pred sparql: ASK WHERE { wd:Q303 wdt:P40 ?uri } 

id: 21
question: Who created English Wikipedia?
ref sparql: SELECT ?uri WHERE { wd:Q52 wdt:P170 ?uri . ?uri wdt:P31 wd:Q5 . }
pred sparql: SELECT DISTINCT ?uri WHERE { wd:Q52 wdt:P112 ?uri . } 

id: 141
question: Which languages are spoken in Estonia?
ref sparql: SELECT * WHERE { wd:Q191 wdt:P2936 ?lang . }
pred sparql: SELECT * WHERE { wd:Q191 wdt:P2936 ?lang . } 

id: 45
question: Where did Abraham Lincoln die

In [22]:
# check ASK queries
count_all = 0
count_ask = 0

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.type == Type.BOOLEAN:
        ques_pair.print(print_answer=True)
        count_all += 1
        if "ASK" in ques_pair.pred_sparql:
            count_ask += 1

print(f'''{count_ask} of {count_all} predicted queries contain "ASK"''')

id: 117
question: Is Pamela Anderson a vegan?
ref sparql: ASK where { wd:Q83325 wdt:P1576 wd:Q83364}
pred sparql: ASK WHERE { wd:Q303 wdt:P40 ?uri } 
ref_answer:
 True
pred_answer:
 True

id: 79
question: Are there any castles in the United States?
ref sparql: ASK WHERE { { ?uri wdt:P31 wd:Q23413. } UNION { ?uri wdt:P31 ?castlesubtype . ?castlesubtype wdt:P279* wd:Q23413 } ?uri wdt:P17 wd:Q30 }
pred sparql: ASK WHERE { ?uri wdt:P31 wd:Q33506 . ?uri wdt:P276 wd:Q84 . } 
ref_answer:
 False
pred_answer:
 True

id: 92
question: Is the wife of President Obama called Michelle?
ref sparql: ASK WHERE { wd:Q76 wdt:P26 ?obj . ?obj rdfs:label ?label FILTER( REGEX( ?label, "Michelle" ) ) }
pred sparql: ASK WHERE { wd:Q76 wdt:P26 ?wife . ?wife rdfs_label ?name . FILTER regex( ?name, "Michelle") } 
ref_answer:
 True
pred_answer:
 []

3 of 3 predicted queries contain "ASK"


In [25]:
# check COUNT queries
count_all = 0
count_count = 0

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.type == Type.COUNT:
        ques_pair.print(print_answer=True)
        count_all += 1
        if "COUNT" in ques_pair.pred_sparql:
            count_count += 1

print(f'''{count_count} of {count_all} predicted queries contain "COUNT"''')

id: 42
question: Which countries have places with more than two caves?
ref sparql: SELECT DISTINCT ?uri WHERE { ?cave wdt:P31 wd:Q35509 ; wdt:P17 ?uri . } GROUP BY ?uri HAVING (COUNT(?cave) > 2)
pred sparql: SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q4022 . ?uri p_P2043 ?stmnode . ?stmnode psv_P2043 ?valuenode . ?valuenode wikibase_quantityAmount ?length . ?valuenode wikibase_quantityUnit ?unit . ?valuenode wikibase_quantityLowerBound ?lowerbound . ?valuenode wikibase_quantityUpperBound ?upperbound .
ref_answer:
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q233'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q236'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q242'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q241'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q252'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34020'}}, {'uri': {'type': 'uri', 'value'

In [24]:
# check RANK queries
count_all = 0
count_contains_order = 0

for ref, pred in zip(ref_ques_list, pred_ques_list):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.type == Type.RANK:
        ques_pair.print(print_answer=True)
        count_all += 1
        if "ORDER" in ques_pair.pred_sparql:
            count_contains_order += 1

print(f'''{count_contains_order} of {count_all} predicted queries contain "ORDER"''')

id: 86
question: What is the highest mountain in Germany?
ref sparql: SELECT ?uri WHERE { ?uri wdt:P31 wd:Q8502 ; wdt:P2044 ?elevation ; wdt:P17 wd:Q183 . } ORDER BY DESC(?elevation) LIMIT 1
pred sparql: SELECT ?res WHERE { ?res wdt:P31 wd:Q8502 . ?res wdt:P131+ wd:Q1202 . ?res wdt:P2044 ?height . } ORDER BY DESC( ?height) LIMIT 1
ref_answer:
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3375'}}]
pred_answer:
 [{'res': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q164561'}}]

id: 49
question: Which frequent flyer program has the most airlines?
ref sparql: SELECT ?frequentflyer WHERE { ?airlines wdt:P31 wd:Q46970 . ?airlines p:P4446/ps:P4446 ?frequentflyer . } GROUP BY ?frequentflyer ORDER BY DESC(COUNT(?airlines)) LIMIT 1
pred sparql: SELECT DISTINCT ?uri WHERE { ?uri wdt:P31 wd:Q46970 } GROUP BY ?uri ORDER BY DESC(COUNT(DISTINCT ?uri) LIMIT 1
ref_answer:
 [{'frequentflyer': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q642970'}}]
pred_answ